# Clustering Crypto

In [9]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [10]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [11]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
# response = requests.get(url).json()
# response['Data']
# crypto_df = pd.DataFrame(response['Data'])
# crypto_df

In [12]:
# Alternatively, use the provided csv file:
#file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv('crypto_data.csv')

# Create a DataFrame
crypto_df


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


### Data Preprocessing

In [13]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.drop(columns=['Unnamed: 0'])
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1247,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [16]:
# Keep only cryptocurrencies that are trading
false_trading = crypto_df[crypto_df['IsTrading']==False].index
crypto_df.drop(false_trading, inplace = True)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [19]:
# Remove the "IsTrading" column
crypto_df.drop(columns=["IsTrading"], inplace = True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [20]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace = True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [21]:
# Remove rows with cryptocurrencies having no coins mined
no_coins = crypto_df[crypto_df['TotalCoinSupply'] == '0'].index
crypto_df.drop(no_coins, inplace = True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
9,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [23]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
CoinName = crypto_df['CoinName']
CoinName = pd.DataFrame(CoinName)
CoinName.head()

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
9,Litecoin


In [24]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns=["CoinName"], inplace = True)
crypto_df.head

<bound method NDFrame.head of         Algorithm ProofType  TotalCoinsMined TotalCoinSupply
0          Scrypt   PoW/PoS     4.199995e+01              42
2          Scrypt   PoW/PoS     1.055185e+09       532000000
5             X13   PoW/PoS     2.927942e+10    314159265359
7         SHA-256       PoW     1.792718e+07        21000000
9          Scrypt       PoW     6.303924e+07        84000000
...           ...       ...              ...             ...
1238      SHA-256      DPoS     2.000000e+09      2000000000
1242       Scrypt   PoW/PoS     1.493105e+07       250000000
1245  CryptoNight       PoW     9.802226e+08      1400222610
1246     Equihash       PoW     7.296538e+06        21000000
1247       Scrypt       PoS     1.283270e+05         1000000

[637 rows x 4 columns]>

In [25]:
# Create dummy variables for text features
crypto_df_encoded = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
crypto_df_encoded.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# Standardize data
data_scaler = StandardScaler()
data_scaler.fit(crypto_df_encoded)
crypto_data_scaled = data_scaler.transform(crypto_df_encoded)
crypto_data_scaled[:5]

array([[-0.09782131, -0.03965512, -0.03965258, -0.03965258, -0.03965258,
        -0.05612135, -0.07949286, -0.03965258, -0.03965258, -0.05612135,
        -0.03965258, -0.03965258, -0.18913582, -0.05612135, -0.03965258,
        -0.03965258, -0.07949286, -0.03965258, -0.08894601, -0.06878853,
        -0.03965258, -0.03965258, -0.17052613, -0.03965258, -0.03965258,
        -0.13856406, -0.03965258, -0.03965258, -0.06878853, -0.03965258,
        -0.03965258, -0.03965258, -0.03965258, -0.06878853, -0.03965258,
        -0.07949286, -0.08894601, -0.07949286, -0.03965258, -0.03965258,
        -0.11971303, -0.13255899, -0.14433757, -0.03965258, -0.05612135,
        -0.03965258, -0.06878853, -0.17052613, -0.03965258, -0.03965258,
        -0.03965258, -0.05612135, -0.18004141, -0.33998043, -0.03965258,
        -0.08894601, -0.06878853, -0.05612135, -0.03965258,  1.42090019,
        -0.06878853, -0.03965258, -0.03965258, -0.06878853, -0.06878853,
        -0.03965258, -0.03965258, -0.03965258, -0.0

### Reducing Dimensions Using PCA

In [27]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_data_scaled)
crypto_pca

array([[-1.39368118e-01, -1.19529245e+00, -4.86527269e-01],
       [-1.24876723e-01, -1.19253473e+00, -4.86790671e-01],
       [ 4.71713203e-01, -1.87434967e+00, -4.92917289e-01],
       ...,
       [-2.48898410e-01,  2.25459239e+00,  3.78916049e-01],
       [-3.92988972e-01,  1.87990149e+00,  3.53143398e-01],
       [-1.93807227e-03, -7.56396410e-01, -2.45486799e-01]])

In [28]:
# Create a DataFrame with the principal components data
df_crypto_pca = pd.DataFrame(crypto_pca, columns = ['Principal Component 1', 'Principal Component 2', 'Principal Component 3'])
df_crypto_pca.head()

,Principal Component 1,Principal Component 2,Principal Component 3
0,-0.139368,-1.195292,-0.486527
1,-0.124877,-1.192535,-0.486791
2,0.471713,-1.874350,-0.492917
3,-0.279620,1.381190,0.185528
4,-0.345776,1.039331,0.002531


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [30]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x='k', y='inertia', xticks=k, title='Elbow Curve')

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [32]:
# Initialize the K-Means model
model = KMeans (n_clusters=5, random_state=0)

# Fit the model
model.fit(df_crypto_pca)

# Predict clusters
predictionas = model.predict(df_crypto_pca)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
df_crypto_pca['class'] = model.labels_

df_crypto_pca

,Principal Component 1,Principal Component 2,Principal Component 3,class
0,-0.139368,-1.195292,-0.486527,1
1,-0.124877,-1.192535,-0.486791,1
2,0.471713,-1.874350,-0.492917,1
3,-0.279620,1.381190,0.185528,0
4,-0.345776,1.039331,0.002531,0
...,...,...,...,...
632,4.222562,0.049682,-0.069110,4
633,-0.139163,-1.195253,-0.486531,1
634,-0.248898,2.254592,0.378916,0
635,-0.392989,1.879901,0.353143,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [41]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    df_crypto_pca,
    x="principal component 1",
    y="principal component 2",
    z="principal component 3",
    color="crypto",
    symbol="crypto",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [35]:
# Table with tradable cryptos
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
9,Scrypt,PoW,6.303924e+07,84000000
...,...,...,...,...
1238,SHA-256,DPoS,2.000000e+09,2000000000
1242,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,CryptoNight,PoW,9.802226e+08,1400222610
1246,Equihash,PoW,7.296538e+06,21000000


In [36]:
# Print the total number of tradable cryptocurrencies
rows = crypto_df.shape[0]
print(rows)

637


#### Scatter Plot with Tradable Cryptocurrencies

In [38]:
# Scale data to create the scatter plot
scaler_min_max = MinMaxScaler()

clustered_df_scaled = scaler_min_max.fit_transform(crypto_df[["TotalCoinSupply", "TotalCoinsMined"]])

In [40]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
scaled_df.hvplot.scatter(x="TotalCoinsMined", 
                            y="TotalCoinSupply", 
                           hover_cols=["CoinName"])